The purpose of this notebook is to document how I clean a dataset in json format.

**Issue to solve**: A key-value pair with multiple values cannot be read by Tableau when the json file was imported.

# NOTE: I wanted to try Pandas for data wrangling and exploratory data analysis, but it's not possible with the datasets being in a JSON format.

In [137]:
import pandas as pd

# Read the report
df = pd.read_json("/kaggle/input/mapakalamidad-ph-api-datasets/archive.json")

# DataFrames' info
print(df.info());
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, type to bbox
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   statusCode  4 non-null      int64 
 1   result      4 non-null      object
dtypes: int64(1), object(1)
memory usage: 96.0+ bytes
None


,statusCode,result
type,200,Topology
objects,200,"{'output': {'type': 'GeometryCollection', 'geo..."
arcs,200,[]
bbox,200,"[118.7348612345, 6.1073926574, 126.0443688272,..."


Notice that the table is horrible. That's why I opted to use the JSON library instead.

In [138]:
import json # data processing

# Importing the data from mapakalamidad.ph's API dataset

Run the cell below to read **current** reports:

**Note:** One type of report at a time.

In [139]:
# CURRENT REPORTS
# Read json file
file = open("/kaggle/input/mapakalamidad-ph-api-datasets/reports.json", "r")

# Read the data
data = file.read()

# Parse the data
reports = json.loads(data)

Run the cell below to read **archival** reports:

**Note:** One type of report at a time.

In [140]:
# ARCHIVAL REPORTS (from January to February only)
# Read json file
file = open("/kaggle/input/mapakalamidad-ph-api-datasets/archive.json", "r")

# Read the data
data = file.read()

# Parse the data
reports = json.loads(data)

#  Step-by-tep process on how the wrangle function was coded

# Checking the reports

In json context, the data is an object. When it was loaded in python it is now read as a dictionary.

Notice that I have to go through a hierarchy of keys (result -> objects -> output -> geometries) in order to access the reports.

Since there are so many reports, I'm going to just check some of it:

In [141]:
count = 0  # counter
for report in reports["result"]["objects"]["output"]["geometries"]:
    count += 1
    print(f"{report}\n")
    if count == 3: # loop will break after the third report
        break

{'type': 'Point', 'properties': {'pkey': '6876', 'created_at': '2023-02-21T14:32:36.888Z', 'source': 'grasp', 'status': 'confirmed', 'url': 'd44b36bc-df05-42e9-a39f-67937ccd5f9c', 'image_url': 'https://images.petabencana.id/d44b36bc-df05-42e9-a39f-67937ccd5f9c.jpg', 'disaster_type': 'flood', 'report_data': {'report_type': 'flood', 'flood_depth': 0}, 'tags': {'district_id': None, 'local_area_id': None, 'instance_region_code': 'PH-00'}, 'title': None, 'text': '#Youth4Bayanihan Jherome Domingo trained me'}, 'coordinates': [121.0099653179, 14.6980055343]}

{'type': 'Point', 'properties': {'pkey': '6875', 'created_at': '2023-02-21T13:59:43.706Z', 'source': 'grasp', 'status': 'confirmed', 'url': '07b6935f-edf3-4dcd-b5fc-37d9cc491d57', 'image_url': None, 'disaster_type': 'flood', 'report_data': {'report_type': 'flood', 'flood_depth': 199}, 'tags': {'district_id': None, 'local_area_id': None, 'instance_region_code': 'PH-40'}, 'title': None, 'text': '#Youth4Bayanihan Maria Christina Labrador tr

There are many keys in the report, but the `coordinates` key is what I should focus on. The problem is that Tableau won't read it because it contains a list.

Note that Tableau only accepts single-valued attribute (since the coordinates contain a list, it will only read the first value).

# Cleaning the data

Basically, I just have to separate the coordinates into latitude and longitude.

Note: One of my friends cleaned the data manually (*he modified the contents of the file directly*.) I thought that it was tedious and I wanted to make it easier using the json library for future automation of cleaning the said reports.

In [142]:
## Sample of how it should be done

# Get one sample of report
sample_report = dict(reports["result"]["objects"]["output"]["geometries"][0])

# Get the latitude and longitude
if "coordinates" in sample_report.keys():
    lat, lon = sample_report["coordinates"]
print(f"{lat}, {lon}")

121.0099653179, 14.6980055343


Now that I have the latitude and longitude, I won't be needing the coordinates key.

In [143]:
# Removing the coordinates key
print(f"Before removing the coordinate:\n{sample_report}\n")
if "coordinates" in sample_report.keys():
    sample_report.pop("coordinates")
print(f"After removing the coordinate:\n{sample_report}\n")

Before removing the coordinate:
{'type': 'Point', 'properties': {'pkey': '6876', 'created_at': '2023-02-21T14:32:36.888Z', 'source': 'grasp', 'status': 'confirmed', 'url': 'd44b36bc-df05-42e9-a39f-67937ccd5f9c', 'image_url': 'https://images.petabencana.id/d44b36bc-df05-42e9-a39f-67937ccd5f9c.jpg', 'disaster_type': 'flood', 'report_data': {'report_type': 'flood', 'flood_depth': 0}, 'tags': {'district_id': None, 'local_area_id': None, 'instance_region_code': 'PH-00'}, 'title': None, 'text': '#Youth4Bayanihan Jherome Domingo trained me'}, 'coordinates': [121.0099653179, 14.6980055343]}

After removing the coordinate:
{'type': 'Point', 'properties': {'pkey': '6876', 'created_at': '2023-02-21T14:32:36.888Z', 'source': 'grasp', 'status': 'confirmed', 'url': 'd44b36bc-df05-42e9-a39f-67937ccd5f9c', 'image_url': 'https://images.petabencana.id/d44b36bc-df05-42e9-a39f-67937ccd5f9c.jpg', 'disaster_type': 'flood', 'report_data': {'report_type': 'flood', 'flood_depth': 0}, 'tags': {'district_id': No

Since the coordinate is gone, I will store the latitude and longitude so that I still have access to the geolocation of the report.

In [144]:
# Storing the lat and lon keys in the report
sample_report["lat"] = lat
sample_report["lon"] = lon
sample_report

{'type': 'Point',
 'properties': {'pkey': '6876',
  'created_at': '2023-02-21T14:32:36.888Z',
  'source': 'grasp',
  'status': 'confirmed',
  'url': 'd44b36bc-df05-42e9-a39f-67937ccd5f9c',
  'image_url': 'https://images.petabencana.id/d44b36bc-df05-42e9-a39f-67937ccd5f9c.jpg',
  'disaster_type': 'flood',
  'report_data': {'report_type': 'flood', 'flood_depth': 0},
  'tags': {'district_id': None,
   'local_area_id': None,
   'instance_region_code': 'PH-00'},
  'title': None,
  'text': '#Youth4Bayanihan Jherome Domingo trained me'},
 'lat': 121.0099653179,
 'lon': 14.6980055343}

# Finally, the sample report is ready for analysis!

# Creating a wrangle function

I have two api sources that I'm going to used:
1. **. Current reports**
2. **. Archival reports**

So, I'm going to create a function to make the process of cleaning the data from both sources wouldn't be repetitive.

The function will read and then clean the data. After the process is applied, it will save the reports in a separate file ready for visualization.

In [145]:
def wrangle(reports):
    for report in reports["result"]["objects"]["output"]["geometries"]:
        # Get the latitude and longitude from the coordinate of the report
        lat, lon = report["coordinates"]
        
        # Drop the coordinates key from the report
        report.pop("coordinates")
        
        # Store the latitude and longitude keys in the report
        report["lat"] = lat
        report["lon"] = lon
        
    # Saving the reports in a new file
    file_path = '/kaggle/working/cleaned.json'
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(reports, file, ensure_ascii=False, indent=4)
    
    # Display the first three cleaned reports
    count = 0  # counter
    print("The first three reports from the dataset:")
    for report in reports["result"]["objects"]["output"]["geometries"]:
        count += 1
        print(f"{report}\n")
        if count == 3: # loop will break after the third report
            break
    
    return reports

I'll overwrite the reports with the archival ones. Let's try the wrangle function with it:

In [146]:
reports = wrangle(reports)

The first three reports from the dataset:
{'type': 'Point', 'properties': {'pkey': '6876', 'created_at': '2023-02-21T14:32:36.888Z', 'source': 'grasp', 'status': 'confirmed', 'url': 'd44b36bc-df05-42e9-a39f-67937ccd5f9c', 'image_url': 'https://images.petabencana.id/d44b36bc-df05-42e9-a39f-67937ccd5f9c.jpg', 'disaster_type': 'flood', 'report_data': {'report_type': 'flood', 'flood_depth': 0}, 'tags': {'district_id': None, 'local_area_id': None, 'instance_region_code': 'PH-00'}, 'title': None, 'text': '#Youth4Bayanihan Jherome Domingo trained me'}, 'lat': 121.0099653179, 'lon': 14.6980055343}

{'type': 'Point', 'properties': {'pkey': '6875', 'created_at': '2023-02-21T13:59:43.706Z', 'source': 'grasp', 'status': 'confirmed', 'url': '07b6935f-edf3-4dcd-b5fc-37d9cc491d57', 'image_url': None, 'disaster_type': 'flood', 'report_data': {'report_type': 'flood', 'flood_depth': 199}, 'tags': {'district_id': None, 'local_area_id': None, 'instance_region_code': 'PH-40'}, 'title': None, 'text': '#Yout

It works! The cleaned reports are now in the /kaggle/working directory named "cleaned.json".